In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import time
import csv
from sklearn.inspection import permutation_importance

In [2]:
def find_important_feature(model,file_numb):
    df = pd.read_csv(r'C:\Users\johnk\OneDrive\Desktop\project\python project\kaggle\amex-default-prediction\new_train\train_{0}.csv'.format(file_numb))
    value_counts = pd.DataFrame(list(df.describe().iloc[0]), columns = ['counts'])
    col = pd.DataFrame(df.describe().iloc[0].index, columns = ['col'])
    df_count = pd.concat([col,value_counts], axis=1)
    keep = df_count.loc[df_count['counts']>1000].iloc[:-1]
    feature_name = list(keep['col'])
    label = pd.DataFrame(df['target'], columns=['target'])
    np_label =  np.array(label).T[0]
    for y in feature_name:
        df[y]=df[y].fillna(df[y].mean())
    feature = df[feature_name]
    if model == LogisticRegression:
        mdl = model(max_iter =20**9)
    else: 
        mdl = model()
    mdl.fit(feature, np_label)
    feature_importance = permutation_importance(mdl, feature, np_label, n_repeats=10)
    fi = pd.DataFrame(feature_importance.importances_mean, index = feature.columns, columns = ['importance'])
    important_l1= fi.loc[fi['importance']>0].index
    return important_l1

In [3]:
def update_important_l1(model, file_number):
    i = pd.read_csv('important_l1.csv')
    important_l1 = find_important_feature(model, file_number)
    if model == GradientBoostingClassifier:
        for x in important_l1:
            i.loc[0,x]+=1
    if model == LogisticRegression:
        for x in important_l1:
            i.loc[1,x]+=1
    i.to_csv('important_l1.csv', index = False)

In [4]:
models = [GradientBoostingClassifier]

In [5]:
for x in range(0,20):
    start_time = time.time()
    for m in models:
        important_l1 = update_important_l1(m,x)
        print('. ', end='')
    print(x, end=' ')
    print(" %s second " %(time.time()-start_time))

. 0  4099.494049549103 second 
. 1  3040.5778987407684 second 
. 2  2607.886709690094 second 
. 3  4099.566225528717 second 
. 4  2488.1827986240387 second 
. 5  2545.2422466278076 second 
. 6  2511.7177183628082 second 
. 7  2555.4650416374207 second 
. 8  2192.2873837947845 second 
. 9  2037.8780126571655 second 
. 10  2146.6922466754913 second 
. 11  2100.353888988495 second 
. 12  2060.1722996234894 second 
. 13  1996.2748103141785 second 
. 14  2230.5583822727203 second 
. 15  2175.1310291290283 second 
. 16  2047.5375006198883 second 
. 17  1986.2682464122772 second 
. 18  1924.280023097992 second 
. 19  2093.7526111602783 second 
